In [1]:
%matplotlib inline

import matplotlib as mpl

import os
import numpy as np
import matplotlib.pyplot as plt

from BQ_code.utils import FUNC, ALGO
from BQ_code.functions import *

### Error Calculation

In [2]:
NUM_TRIAL = 1
NUM_EXP = 1
MAX_STEP= 250

In [3]:
def read_calc_error(func_name, noise, algo_name, split=None):
    if split == None:
        result_dir = os.path.join(base_dir, FUNC[func_name], str(noise), ALGO[algo_name])
    else:
        result_dir = os.path.join(base_dir, FUNC[func_name], str(noise), str(split), ALGO[algo_name])
    
    datadir = os.path.join(result_dir, func_name + '_' + algo_name +'_query_history.npy')
    history_quad = np.load(datadir, allow_pickle=True)[()]['integrals']
    
    pad_num = MAX_STEP - len(history_quad[0])
    history_quad = np.pad(history_quad, [(0,0),(pad_num,0)], 'constant')

    history_quad = history_quad.reshape(-1, NUM_EXP, MAX_STEP)
    quad_mean = np.mean(history_quad, axis=1)
    
    true_quad = eval(FUNC[func_name])().true_quad
    error = np.abs(true_quad - quad_mean)
    
    error_mean = np.mean(error, axis=0)
    error_std = np.std(error, axis=0)
    
    return error_mean, error_std

### Color specification

In [4]:
plt.style.use('default')
import seaborn as sns

colors = sns.color_palette("dark")
greens = sns.color_palette("BuGn", 10)
blues = sns.color_palette("PuBu", 10)
reds = sns.color_palette("YlOrRd", 10)
purples = sns.color_palette("PuOr", 10)

FMT = {
        'mvs-mat' : '>',
        'mvs-mc-mat': '>',
        'mvs-se' : 's',
        'mvs-mc-se': 's',
        'mc' : 'h',
        'trap' : 'd',
        }

COLOR = {
        'mvs-mat' : reds[5],
        'mvs-mc-mat': reds[9],
        'mvs-se' : blues[9],
        'mvs-mc-se' : purples[9],
        'mc' : greens[9],
        'trap' : blues[9],
        }

LINE = {
        'mvs-mat' : '-',
        'mvs-mc-mat': '-',
        'mvs-se' : '-',
        'mvs-mc-se': '-',
        'mc' : '-',
        'trap' : '-',
        }

### Error plot

In [5]:
BASE = 15
SPLIT = int(MAX_STEP // BASE)
base_dir = './results'

def plot(func_name, noise, algo_list, loc='upper right', yrange=None, std_scale=None):
    """
    Function for plotting the results
    """
    fig, ax = plt.subplots(figsize=(4.5,3),dpi=150)
    std_scale = np.ones(len(algo_list)) if std_scale==None else std_scale
    
    plt_list = []
    label_list = []
    
    for i, algo_name in enumerate(algo_list):
        error_mean, error_std = read_calc_error(func_name, noise, algo_name)
        error_std *= std_scale
        if algo_name == 'trap':
            total_square = int(MAX_STEP ** 0.5 + 1e-8)
            error_mean = error_mean[::int(MAX_STEP // total_square)]
            error_std = error_std[::int(MAX_STEP // total_square)]
            xs = np.arange(total_square+1)**2
        else:
            error_mean = np.append(error_mean[::BASE], error_mean[-1]) 
            error_std = np.append(error_std[::BASE], error_std[-1]) 
            xs = np.arange(0,MAX_STEP+BASE,BASE)
        
        line_plt, = ax.plot(xs, error_mean, linestyle=LINE[algo_name], lw=0.8, color=COLOR[algo_name])
        
        band_plt = ax.fill_between(xs, error_mean-error_std, error_mean+error_std, \
                        color=COLOR[algo_name],linestyle=LINE[algo_name],\
                       lw=0.5, alpha=0.2)
        
        plt_list.append((line_plt, band_plt))
        label_list.append(ALGO[algo_name])
        
    
    ax.legend(plt_list, label_list, loc=loc, fontsize=10)

    ax.set_xlabel('Queries', fontsize=13)
    ax.set_ylabel('Error', fontsize=13)
    ax.set_yscale('log')
    if yrange is not None:
        ax.set_ylim(*yrange)
    ax.set_title(FUNC[func_name] + ' - $\sigma='+str(noise)+'$', fontsize=14)
    plt.show()

    ### Uncomment to save figure
    # fig.savefig(base_dir + '\\%s-%s.pdf'%(func_name,noise), bbox_inches='tight') 
    

### Plot Test Function

In [ ]:
plot('ack1', 0, [ 'mc','mvs-mat', 'mvs-mc-mat', 'mvs-se', 'mvs-mc-se'], 'upper right', std_scale=0.5)

### Split Plot

In [6]:
NUM_TRIAL = 10
NUM_EXP = 1
base_dir = './results-split/'
SPLITS = [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1.0]

In [7]:
def plot_split(func_name, noise, algo_list, split_list, loc='upper right', std_scale=1.):
    """
    Function for plotting the results
    """
    fig, ax = plt.subplots(figsize=(4.5,3),dpi=150)
    
    for algo_name in algo_list:
        error_mean = []
        error_std = []
        for split in split_list:
            mean, std = read_calc_error(func_name, noise, algo_name, split)
            error_mean.append(mean[-1])
            error_std.append(std[-1])

        error_mean = np.array(error_mean)
        error_std = np.array(error_std)

        ax.errorbar(range(len(split_list)), error_mean, error_std*std_scale,\
                     fmt=FMT[algo_name], color=COLOR[algo_name], label=ALGO[algo_name],\
                    linestyle=LINE[algo_name], capsize=4, lw=1.5, markersize=5)
    
    ax.legend(loc=loc, fontsize=10)
    ax.set_xticks(range(len(split_list)))
    ax.set_xticklabels(split_list)
    ax.set_xlabel('MVS-MC Splits', fontsize=13)
    ax.set_ylabel('Error', fontsize=13)
    ax.set_title(FUNC[func_name] + ' - $\sigma='+str(noise)+'$', fontsize=14)
    plt.show()
    
    ### Uncomment to save figure
    # fig.savefig(base_dir + 'split-%s-%s.pdf'%(func_name,noise), bbox_inches='tight') 

In [ ]:
plot_split('ack2', 0, ['mvs-mc-mat', 'mvs-mc-se'], SPLITS, 'upper right', 0.3)